In [1]:
!wget -nc https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!unzip -qq flower_data.zip

--2020-08-21 15:55:25--  https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.83.26
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.83.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 311442766 (297M) [application/zip]
Saving to: ‘flower_data.zip’

flower_data.zip     100%[===================>] 297.01M  16.8MB/s    in 20s     

2020-08-21 15:55:45 (15.2 MB/s) - ‘flower_data.zip’ saved [311442766/311442766]



In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time


In [3]:
from torch.utils.data import Dataset, DataLoader

In [4]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [5]:
train_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.CenterCrop(224),
                                       transforms.RandomRotation(30),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


In [6]:
train_data = datasets.ImageFolder(root = '/content/flower_data/train',transform=train_transforms)

In [7]:
VALID_RATIO = 0.25

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples
print(n_train_examples)
print(n_valid_examples)

train_data, valid_data = data.random_split(train_data, 
                                           [n_train_examples, n_valid_examples])

1638
4914


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
import copy
import random
import time
import torchvision.models as models

In [10]:
model = models.resnet101(pretrained=True).to(device)

for param in model.parameters():
  param.requires_grad=True
    
model.fc = nn.Sequential(
               nn.Linear(2048, 496),
               nn.ReLU(inplace=True),
               nn.Linear(496, 102)).to(device)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


In [11]:
BATCH_SIZE = 16
train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data, 
                                 batch_size = BATCH_SIZE)

In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 43,567,158 trainable parameters


In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, dampening=0, weight_decay=0.0001, nesterov=False)

criterion = criterion.to(device)

In [14]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [15]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        y = y.type(torch.LongTensor)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred = model(x)
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [16]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)
            y = y.type(torch.LongTensor)
            y = y.to(device)

            y_pred = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [17]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [18]:
EPOCHS = 200

best_valid_loss = float('inf')

for epoch in range(EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    if valid_loss < best_valid_loss :
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 3m 21s
	Train Loss: 3.853 | Train Acc: 16.32%
	 Val. Loss: 2.711 |  Val. Acc: 37.68%
Epoch: 02 | Epoch Time: 3m 21s
	Train Loss: 2.312 | Train Acc: 41.59%
	 Val. Loss: 2.383 |  Val. Acc: 42.53%
Epoch: 03 | Epoch Time: 3m 20s
	Train Loss: 1.689 | Train Acc: 55.58%
	 Val. Loss: 1.945 |  Val. Acc: 50.79%
Epoch: 04 | Epoch Time: 3m 20s
	Train Loss: 1.496 | Train Acc: 58.09%
	 Val. Loss: 2.210 |  Val. Acc: 52.44%
Epoch: 05 | Epoch Time: 3m 20s
	Train Loss: 1.309 | Train Acc: 64.44%
	 Val. Loss: 2.098 |  Val. Acc: 51.64%
Epoch: 06 | Epoch Time: 3m 20s
	Train Loss: 1.061 | Train Acc: 70.23%
	 Val. Loss: 1.487 |  Val. Acc: 62.24%
Epoch: 07 | Epoch Time: 3m 20s
	Train Loss: 0.958 | Train Acc: 72.29%
	 Val. Loss: 1.125 |  Val. Acc: 71.12%
Epoch: 08 | Epoch Time: 3m 21s
	Train Loss: 0.878 | Train Acc: 75.36%
	 Val. Loss: 2.400 |  Val. Acc: 53.96%


KeyboardInterrupt: ignored